<a href="https://colab.research.google.com/github/rckormos/PyCCCP/blob/main/CCCP_Fit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Fitting of Coiled Coil Parameters to PDB Structures

User-friendly tools for fitting coiled coil protein structures to an input structure using the Crick parameterization as described in [Grigoryan and DeGrado, 2011](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3052747). For more details and the source code, please refer to the accompanying [GitHub repository](https://github.com/rckormos/PyCCCP).

In [1]:
#@title Input settings for the fitter below, then hit `Runtime` -> `Run all`

#@markdown ###<ins>**General Parameters**</ins>
#@markdown **PDB Structure**
pdb = "" #@param {type:"string"}
Assembly = False #@param {type:"boolean"}
#@markdown - If the "pdb" field is empty, you will be prompted to load a PDB 
#@markdown   file from your local machine. If the "Assembly" box is checked, 
#@markdown   and the "pdb" field is not empty, the biological assembly will be
#@markdown   used instead of the asymmetric unit.


#@markdown **Selection**
selection = "A7-32:39-64:84-109:116-141" #@param {type:"string"}
#@markdown - If empty, the entire structure specified in the "pdb" field will 
#@markdown   be aligned against. Otherwise, a selection of residues should be 
#@markdown   provided such that discontiguous fragments are separated by colons 
#@markdown   and a one-letter chain identifier is provided at the beginning of 
#@markdown   each fragment that begins a new chain. For example, 
#@markdown   "A7-32:39-64:B7-32:39-64" would specify residues 7 to 32 
#@markdown   (inclusive) and 39 to 64 from both chains A and B of a structure.

#@markdown **Number of Chains**
cN = 4 #@param {type:"number"}
#@markdown **Backbone Type**
bbtype = 'ala' #@param ['ca', 'gly', 'ala'] {type:"string"}
#@markdown **Job Name**
jobname = 'full_ferritin_test' #@param {type:"string"}
#@markdown - This parameter will be used to generate output file names along 
#@markdown   with the job hash.

#@markdown ###<ins>**Parameterization Type**</ins>
pType = 'GENERAL' #@param ['GENERAL', 'SYMMETRIC', 'ZOFF-SYMM', 'DPH0-SYMM', 'GENERAL-HLXPH', 'SYMMETRIC-HLPXH', 'ZOFF-SYMM-HLXPH'] {type:"string"}
#@markdown - 'GENERAL' -- We recommend trying this setting first. It assumes 
#@markdown   nothing about the symmetry of the structure, allowing each chain 
#@markdown   to have its own minor helical phase, superhelical phase offset and 
#@markdown   Z-offset. By analyzing the best-fit parameters, one can determine 
#@markdown   whether the structure actually has any kind of symmetry.
#@markdown - 'GENERAL-HLXPH' -- Same as GENERAL, but all chains share the same 
#@markdown   value of minor helical phase.
#@markdown - 'SYMMETRIC' -- At most one Z-offset and one variable superhelical 
#@markdown   phase offset is allowed, which relate all up chains to all down 
#@markdown   chains. For parallel topologies this results in Cn symmetry (n is 
#@markdown   the number of chains). For alternating up/down topologies, this 
#@markdown   results in Dn symmetry. And for all other topologies super-helical 
#@markdown   phase offset between adjacent chains is set to 2π/n and only one 
#@markdown   Z-offset is allowed for all up chains relative to all down chains.
#@markdown - 'SYMMETRIC-HLXPH' -- Same as SYMMETRIC, but all chains share the 
#@markdown   same value of minor helical phase.
#@markdown - 'ZOFF-SYMM' -- Same as GENERAL, but only one z-offset is allowed 
#@markdown   (i.e. all up chains are offset relative to all down chains by the 
#@markdown   same amount).
#@markdown - 'ZOFF-SYMM-HLXPH' -- Same as ZOFF-SYMM, but all chains share the 
#@markdown   same value of minor helical phase.
#@markdown - 'DPH0-SYMM' -- Same as GENERAL, but superhelical phase symmetry 
#@markdown   is imposed.

#@markdown ###<ins>**Initial Values**</ins>
Seeded = False #@param {type:"boolean"}
#@markdown - If the "Seeded" box is checked, please specify in the fields below 
#@markdown   a starting point for the optimizer that you think might be close 
#@markdown   to a good solution if you get an unreasonable solution otherwise 
#@markdown   (and you think better solutions exist).

#@markdown **Superhelical Radius, $r_0$ (Ångstroms)**
r0 = 5.0 #@param {type:"number"}
#@markdown **Minor Helical Radius, $r_1$ (Ångstroms)**
r1 = 2.26 #@param {type:"number"}
#@markdown **Superhelical Frequency, $\omega_0$ (Degrees / Residue)**
w0 = -3.6 #@param {type:"number"}
#@markdown **Minor Helical Frequency, $\omega_1$ (Degrees / Residue)**
w1 = 102 #@param {type:"number"}
#@markdown **Superhelical Pitch Angle, $\alpha$ (Degrees)**
a = -12 #@param {type:"number"}
#@markdown **Minor Helical Phase (for all chains), $\phi_1$ (Degrees)**
ph1 = 0 #@param {type:"number"}

In [2]:
#@title Install dependencies
%%bash -s $python_version

set -e

PYTHON_VERSION=$1

if [ ! -f PYCCCP_READY ]; then
  echo "installing pycccp..."
  # install dependencies
  pip install -q "pycccp @ git+https://github.com/rckormos/PyCCCP"
  touch PYCCCP_READY
fi

installing pycccp...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00


In [3]:
#@title Fit ideal coiled coil
import os
from pprint import pprint
from google.colab import files

import numpy as np
from Bio.PDB.PDBIO import PDBIO

from pycccp.PDBIO import coordsToPDB
from pycccp.fitCrickBB import fitCrickBB

if not os.path.exists(f'{jobname}_result'):
  os.mkdir(f'{jobname}_result')

if Seeded:
    IP = [r0, r1, w0, w1, a, ph1]
else:
    IP = []

def get_pdb(pdb_code=None, assembly=False):
    """Code written by Sergey Ovchinnikov for PDB file downloading.

    Parameters
    ----------
    pdb_code : str or None
        Four-letter code for a PDB file to download.  If None is provided, 
        the user will be prompted to upload a PDB file.

    Returns
    -------
    pdbpath : str
        Path to downloaded or uploaded PDB file.
    """
    if pdb_code is None or pdb_code == "":
        upload_dict = files.upload()
        pdb_string = upload_dict[list(upload_dict.keys())[0]]
        with open("tmp.pdb","wb") as out:
             out.write(pdb_string)
        return "tmp.pdb"
    elif os.path.isfile(pdb_code):
        return pdb_code
    elif len(pdb_code) == 4:
        if assembly:
            os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb1")
            return f"{pdb_code}.pdb"
        else:
            os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb1")
            return f"{pdb_code}.pdb"
    else:
        os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
        return f"AF-{pdb_code}-F1-model_v3.pdb"

pdbfile = get_pdb(pdb)

params_dict, err, xyz, chains, structure = fitCrickBB(pdbfile, cN, pType, IP, 
                                                      selection=selection, 
                                                      bbtype=bbtype)

print('RMSD = {} Angstroms'.format(err))
pprint(params_dict, sort_dicts=False)

ideal_pdb_path = f'{jobname}_result/' + jobname + '_ideal.pdb'
aligned_pdb_path = f'{jobname}_result/' + jobname + '_aligned.pdb'

coordsToPDB(xyz, chains, ideal_pdb_path, bbtype)

io = PDBIO()
io.set_structure(structure)
io.save(aligned_pdb_path)

Saving 1jgc.pdb to 1jgc.pdb


/usr/local/lib/python3.9/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1279
  warnings.warn(


RMSD = 1.1814565730522888 Angstroms
{'cN': 4,
 'chL': 26,
 'r0': 7.600448255231103,
 'r1': 2.260348720680082,
 'w0': -2.284559127331303,
 'w1': 101.62364644864809,
 'a': -11.658182769500943,
 'pitch': 231.450849761228,
 'cr': [0.0, 0.0, 1.0],
 'ph1': [160.8660227765913,
         59.42763010403005,
         264.9515795056496,
         165.67341363376102],
 'phCa': [27.360608571183743,
          27.545862347270507,
          29.822518851593884,
          32.167999428353376],
 'dph0': [176.05066707153236, 340.2155028096133, 164.1607139002561],
 'zoff_apNN': [-0.9154916901968804, -0.6807807783201127, 2.1448823506978516],
 'zoff_register': [-2.037531415375595, 0.2777648976097158, 2.075400185969816],
 'zoff_aa': [2.9130439061702855, -5.136339932710031, 2.1116241013036077],
 'starting_heptad_pos': ['d', 'c', 'e', 'd']}


In [4]:
#@title Visualize coils (reference in red, ideal fit in blue)
import py3Dmol

view = py3Dmol.view(width=400, height=300)

for pdb_path in [ideal_pdb_path, aligned_pdb_path]:
  with open(pdb_path) as ifile:
    system = "".join([x for x in ifile])
  view.addModelsAsFrames(system)

view.setStyle({'model': 0}, {"cartoon": {'color': 'red'}, 
                              "stick": {'color': 'spectrum'}})
view.setStyle({'model': -1}, {"cartoon": {'color': 'blue'}, 
                              "stick": {'color': 'spectrum'}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
#@title Package and download results
import shutil
from google.colab import files

shutil.make_archive(f'{jobname}.result', 'zip', f'{jobname}_result')
files.download(f'{jobname}.result.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>